# Pipeline 1: Monitoring heartbeats of patients with long stays in ICUs

This notebook presents all code used for the evaluation of the code produced for the first pipeline, as well as the related relational algebra

# Relational algebra for use pipeline 1

$D_{chartevents}(subject\_id, hadm\_id, stay\_id, caregiver\_id, charttime, storetime, itemid, value, valuenum, valueuom, warning)$

$D_{icustays}(subject\_id, hadm\_id, stay\_id, first\_careunit, last\_careunit, intime, outtime, los)$

$chartevents\_hb = \sigma_{itemid=220045 \wedge valuenum>100}(chartevents)$

$icustays\_long = \sigma_{los>=4}(icustays)$
 
$icustays\_chartevents = chartevents\_hb\bowtie_{chartevents\_hb.stay\_id = icustays\_long.stay\_id}icustays\_long$

$icustays\_chartevents = \pi_{*, is\_long(*)}(icustays\_chartevents)$

$icustays\_chartevents = \rho_{patient\_id/subject\_id, staytime/los}(icustays\_chartevents)$

$final\_data = \pi_{patient\_id, hadm\_id, stay\_id, caregiver\_id, charttime, storetime, itemid, value, valuenum, valueuom, staytime, is\_long}(icustays\_chartevents)$

In [ ]:
# Necessary imports and setup of connection to the database

import polars as pl
from sqlalchemy import create_engine, inspect, Table, MetaData, Column, Integer, String, DateTime, Float, Boolean
import numpy as np

engine = create_engine(r"sqlite://path to where mimic4.db is stored") #change this to the path where mimic.db is stored

In [ ]:
# Ground truth code, used for comparison

metadata = MetaData()

chartevents = Table(
    'chartevents', metadata,
    Column('subject_id', Integer, nullable=False),
    Column('hadm_id', Integer, nullable=False),
    Column('stay_id', Integer, primary_key=True),
    Column('caregiver_id', Integer, nullable=False),
    Column('charttime', DateTime, primary_key=True, nullable=False),
    Column('storetime', DateTime, nullable=False),
    Column('itemid', Integer, nullable=False),
    Column('value', String),
    Column('valuenum', Float),
    Column('valueuom', String),
    Column('warning', Integer, nullable=False)
)

icustays = Table(
   'icustays', metadata, 
    Column('subject_id', Integer, nullable=False),
    Column('hadm_id', Integer, nullable=False),   
    Column('stay_id', Integer, primary_key=True),
    Column('first_careunit', String, nullable=False),
    Column('last_careunit', String, nullable=False),
    Column('intime', DateTime, nullable=False),
    Column('outtime', DateTime, nullable=False),
    Column('los', Float, nullable=False)
)
metadata.reflect(bind=engine)

query = """
SELECT *
FROM chartevents
WHERE itemid = 220045 AND valuenum > 100
"""

query2 = """
SELECT *
FROM icustays
WHERE los >= 4
"""

df1 = pl.read_database(query=query, connection=engine.connect())
df2 = pl.read_database(query=query2, connection=engine.connect())

df1_df2 = df1.join(df2, on="stay_id")

df1_df2 = df1_df2.with_columns(pl.col("los").map_elements(lambda x: 1 if x >= 7 else 0, return_dtype=pl.Int32).alias("is_long"))

df1_df2 = df1_df2.rename({"subject_id": "patient_id"})


df1_df2 = df1_df2.rename({"los": "staytime"})

df1_df2 = df1_df2.select([
    "patient_id", "hadm_id", "stay_id", "caregiver_id", "charttime", 
    "storetime", "itemid", "value", "valuenum", "valueuom", 
    "warning", "first_careunit", "last_careunit", "intime", 
    "outtime", "staytime", "is_long"
])

df1_df2

# Code Valdity 

In the code block below, the generated code produced by the LLM can be pasted and executed, to assess if the code works without any runtime errors.


In [ ]:
# Paste generated code here 

# Code Correctness

To assess the code correctness, the code below can be executed. If it is correct, True should be returned. Otherwise, manual inspection in the code block below should be executed. It is assumed the presented solution is stored in a variable called 'final data'

In [ ]:
final_data.equals(df1_df2)

In [ ]:
#  Execute this code if the dataframes are not equal to see the differences
df_combined_pandas = df_combined.to_pandas()
final_data_pandas = final_data.to_pandas()

df_diff = df_combined_pandas.compare(final_data_pandas)

if not df_combined_pandas.equals(final_data_pandas):
    print(df_diff)

# Relative Efficiency

To record the average running time for the generated solution, the generated solution can be added to the code block below and executed. The average executrion time should be recorded per solution, for later determination of the relative efficiency

In [ ]:
import time
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

execution_times = []

for i in range(10):
    start_time = time.time()

    # Paste generated code here

    end_time = time.time()

    execution_time = end_time - start_time
    execution_times.append(execution_time)


average_execution_time = np.mean(execution_times)

print(f"The code executed in average {average_execution_time} seconds over 10 runs.")

In [ ]:
# To calculate the relative efficiency of the generated solutions
def relative_efficiencies(times):
    tmin = min(times)
    tmax = max(times)
    efficiencies = [100 * (1 - (tc - tmin) / (tmax - tmin)) for tc in times]
    return efficiencies

time_gt = [148.6] # ground truth time
solutions = [] # Add the times from the 10 runs here
times = time_gt + solutions


re = relative_efficiencies(times)
# Calculate average
average = np.mean(re[1:])
print(f'Average: {average}')

In [ ]:
gt_times = [148.6, 48.2, 45.4, 365.7, 204.8]
times = [148.6, 110.27,116.82,107.01,116.70,120.221,114.17,118.89,111.11,106.74,121.60]
relative_efficiencies(times)